<h3> In this notebook we will perform preprocessing of the total wine database including</h3>
<ol>
    <li> Creating dummy variables from categorical variables where needed </li>
    <li> Scaling continuous variables</li>
    <li> Split into a training and test test </li>
    <li> Saving the pre-processed and split data into separate CSV files </li>
</ol>

In [1]:
# Load Required Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [156]:
def mlstats(array, labels):
    n = len(array)
    for i in range(n):
        tp = array[i][0,0]
        tn = array[i][1,1]
        fn = array[i][1,0]
        fp = array[i][0,1]
        try:
            prec = round(tp/(tp + fp), 4)
        except:
            prec = np.nan
        try:
            recall = round(tp/(tp+fn), 4)
        except:
            recall = np.nan
        if (fp + tn) != 0:
            spec = round(tn / (fp + tn), 4)
        else:
            spec = 0
        try:
            acc = round((tp + tn) /(tp + tn +fp +fn), 4)
        except:
            acc = np.nan
        try:
            f1 = round(2* prec*recall/(prec + recall), 4)
        except:
            f1 = np.nan
        l = labels[i]

        print(f'Label: {l}, Precision: {prec:.4f}, Recall: {recall:.4f}, Specificity: {spec:.4f}, Accuracy: {acc:.4f}, F1: {f1:.4f} ')
    return

In [2]:
# Load wine database files
wine_qual = pd.read_csv('../data/WineQual.csv')

In [3]:
# Print out head
wine_qual.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,2
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,2


In [4]:
# Create explanatory and predictor variables
x_cols = list(wine_qual.columns)
x_cols.remove('quality')
y = wine_qual['quality']
X = wine_qual[x_cols]

In [5]:
# Create dummies and add back to data table
# From before, red is 2 and white is 1
# to make this a dummy variable, subtract 1 from the wine_color column to make red=1 and white=0
# Quality is an ordinal variable where the higher the number the better the quality
X = pd.get_dummies(X, drop_first = True)
X.loc[:,'wine_color'] -= 1

In [6]:
# Create a stratification column out of y and wine_color
X['strat'] = str(y)+str(X['wine_color'])

In [7]:
# Stratify along y,wine_color for test train split
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify= X['strat'], random_state = 42, \
                                                    test_size=0.20)

In [8]:
# Drop the strat column
X_train = X_train.drop('strat', axis = 1)
X_test = X_test.drop('strat', axis = 1)

In [9]:
# Perform standard scaler on training set and also apply this scaler to test set
# To prevent leakage and because we learn something from the data when we scale, scaling should be done first
# to training data and this scaler applied to the test data.
s = StandardScaler()

scaled_X_train = s.fit_transform(X_train)
df_scaled_X_train = pd.DataFrame(scaled_X_train, index=X_train.index, columns=x_cols)

scaled_X_test = s.fit_transform(X_test)
df_scaled_X_test = pd.DataFrame(scaled_X_test, index=X_test.index, columns=x_cols)


In [10]:
print(df_scaled_X_test.head())

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
2735      -0.568366          0.049829    -0.255062        1.525725  -0.149819   
2538      -1.025779         -0.368288    -1.222375        0.701338  -0.782279   
3283      -0.492130         -0.846136    -1.222375       -0.209827   4.277402   
722       -0.263423         -0.189095     0.159500       -0.817270  -0.782279   
4805      -1.635664         -0.487750     0.159500        0.224061  -0.715704   

      free sulfur dioxide  total sulfur dioxide   density        pH  \
2735             3.170889              1.720949  0.500601 -0.389360   
2538            -0.015010              0.488247  0.237341  0.365739   
3283             0.269445              0.939659 -0.255429 -1.836634   
722             -0.413248              0.245178 -1.018210  0.932064   
4805            -0.242575             -0.258320 -0.076547  0.051114   

      sulphates   alcohol  wine_color  
2735  -0.149138 -0.508151    0.591564  
2538  

<h3>This is the modeling step for the capstone project</h3>
<ol>We will look at several models including:
    <li>Ordinal Regression with various kernals</li>
    <ol>
        <li>Probit</li>
        <li>Logit</li>
        <li>One Customer Kernal</li>
    </ol>
    <li>Tree Regression</li>
    <ol>
        <li>Random Forest Regression</li>
        <li>Other forest methodologies</li>
    </ol>
</ol>

<h3>Load additional required packages</h3>

In [11]:
import scipy.stats as stats
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [17]:
# Logit Model First
mod_prob = OrderedModel(y_train, df_scaled_X_train, distr='logit')
res_prob = mod_prob.fit(method='bfgs', disp =False)
res_prob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                quality   Log-Likelihood:                -5620.5
Model:                   OrderedModel   AIC:                         1.128e+04
Method:            Maximum Likelihood   BIC:                         1.139e+04
Date:                Wed, 20 Apr 2022                                         
Time:                        16:17:35                                         
No. Observations:                5197                                         
Df Residuals:                    5180                                         
Df Model:                          17                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
fixed acidity            0.2930      0.063      4.623      0.000       0.169       0.417
volatile acidity        -0.6697      0.040    -16.934      0.000      -0.747      -0.592
residual sugar           0.8291      0.090      9.202      0.000       0.653       1.006
chlorides               -0.0678      0.035     -1.946      0.052      -0.136       0.000
free sulfur dioxide      0.2665      0.042      6.400      0.000       0.185       0.348
total sulfur dioxide    -0.2432      0.054     -4.506      0.000      -0.349      -0.137
density                 -0.8942      0.142     -6.293      0.000      -1.173      -0.616
pH                       0.2386      0.045      5.336      0.000       0.151       0.326
sulphates                0.2659      0.034      7.909      0.000       0.200       0.332
alcohol                  0.6977      0.070      9.942      0.000       0.560       0.835
wine_color              -0.4294      0.076     -5.686      0.000      -0.577      -0.281
3/4                     -6.3900      0.233    -27.406      0.000      -6.847      -5.933
4/5                      0.8883      0.091      9.781      0.000       0.710       1.066
5/6                      1.1693      0.025     47.161      0.000       1.121       1.218
6/7                      0.9563      0.019     49.144      0.000       0.918       0.994
7/8                      0.8490      0.035     24.488      0.000       0.781       0.917
8/9                      1.6386      0.194      8.451      0.000       1.259       2.019
========================================================================================
"""

<h3>We should drop citric acid as it has a large p-value and is colinear with acidity</h3>

In [19]:
try:
    df_scaled_X_train.drop('citric acid', inplace=True, axis=1)
except:
    print('Citric acid already dropped')

Citric acid already dropped


<h3>Re-run the model</h3>

In [20]:
# Without citric acid
mod_prob = OrderedModel(y_train, df_scaled_X_train, distr='logit')
res_prob = mod_prob.fit(method='bfgs', disp = False)
res_prob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                quality   Log-Likelihood:                -5620.5
Model:                   OrderedModel   AIC:                         1.128e+04
Method:            Maximum Likelihood   BIC:                         1.139e+04
Date:                Wed, 20 Apr 2022                                         
Time:                        16:18:57                                         
No. Observations:                5197                                         
Df Residuals:                    5180                                         
Df Model:                          17                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
fixed acidity            0.2930      0.063      4.623      0.000       0.169       0.417
volatile acidity        -0.6697      0.040    -16.934      0.000      -0.747      -0.592
residual sugar           0.8291      0.090      9.202      0.000       0.653       1.006
chlorides               -0.0678      0.035     -1.946      0.052      -0.136       0.000
free sulfur dioxide      0.2665      0.042      6.400      0.000       0.185       0.348
total sulfur dioxide    -0.2432      0.054     -4.506      0.000      -0.349      -0.137
density                 -0.8942      0.142     -6.293      0.000      -1.173      -0.616
pH                       0.2386      0.045      5.336      0.000       0.151       0.326
sulphates                0.2659      0.034      7.909      0.000       0.200       0.332
alcohol                  0.6977      0.070      9.942      0.000       0.560       0.835
wine_color              -0.4294      0.076     -5.686      0.000      -0.577      -0.281
3/4                     -6.3900      0.233    -27.406      0.000      -6.847      -5.933
4/5                      0.8883      0.091      9.781      0.000       0.710       1.066
5/6                      1.1693      0.025     47.161      0.000       1.121       1.218
6/7                      0.9563      0.019     49.144      0.000       0.918       0.994
7/8                      0.8490      0.035     24.488      0.000       0.781       0.917
8/9                      1.6386      0.194      8.451      0.000       1.259       2.019
========================================================================================
"""

In [21]:
try:
    df_scaled_X_test.drop('citric acid', inplace = True, axis = 1)
except:
    print('Column citric acid already dropped')
    
predicted = res_prob.model.predict(res_prob.params, exog = np.array(df_scaled_X_test)[:, None])


Column citric acid already dropped


In [98]:
rowmax = np.amax(predicted[:,0], axis = 1)

In [99]:
# Need to change back to 3 to 9 scale
y_pred = []
for i, r in enumerate(rowmax):
    y_pred.append(list(predicted[i,0]).index(r)+3)
    

In [100]:
# Using the list comparison what is the percentage of matches
np.sum(y_test==y_pred)/len(y_test)

0.5292307692307693

In [120]:
# Let's look at the multiclass confusion matrix
from sklearn.metrics import multilabel_confusion_matrix

In [121]:
ml = multilabel_confusion_matrix(y_test, y_pred)
print(ml)
l =['3', '4', '5', '6', '7', '8', '9']
mlstats(ml, l)

array([[[1289,    0],
        [  11,    0]],

       [[1260,    0],
        [  39,    1]],

       [[ 720,  166],
        [ 175,  239]],

       [[ 342,  379],
        [ 163,  416]],

       [[1027,   57],
        [ 175,   41]],

       [[1263,    1],
        [  36,    0]],

       [[1296,    0],
        [   4,    0]]], dtype=int64)

In [111]:
# Some room for improvement,  several of the acid measures are redundent, 
# let's drop them. And clorides is not significant

try:
    df_scaled_X_train.drop(['chlorides', 'fixed acidity'], inplace = True, axis = 1)
except:
    print('Columns from X_train already dropped')
try:
    df_scaled_X_test.drop(['chlorides', 'fixed acidity'], inplace = True, axis = 1)
except:
    print('Columns from X_test already dropped')


Columns X_train already dropped
Column from X_test already dropped


In [112]:
mod_prob = OrderedModel(y_train, df_scaled_X_train, distr='logit')
res_prob = mod_prob.fit(method='bfgs', disp = False)
res_prob.summary()    

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                quality   Log-Likelihood:                -5634.8
Model:                   OrderedModel   AIC:                         1.130e+04
Method:            Maximum Likelihood   BIC:                         1.140e+04
Date:                Wed, 20 Apr 2022                                         
Time:                        18:10:31                                         
No. Observations:                5197                                         
Df Residuals:                    5182                                         
Df Model:                          15                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
volatile acidity        -0.7067      0.039    -18.171      0.000      -0.783      -0.631
residual sugar           0.5320      0.058      9.209      0.000       0.419       0.645
free sulfur dioxide      0.2620      0.042      6.305      0.000       0.181       0.343
total sulfur dioxide    -0.2593      0.054     -4.836      0.000      -0.364      -0.154
density                 -0.3858      0.078     -4.968      0.000      -0.538      -0.234
pH                       0.0962      0.030      3.243      0.001       0.038       0.154
sulphates                0.2254      0.032      6.939      0.000       0.162       0.289
alcohol                  0.9324      0.048     19.501      0.000       0.839       1.026
wine_color              -0.3219      0.070     -4.597      0.000      -0.459      -0.185
3/4                     -6.3863      0.233    -27.376      0.000      -6.844      -5.929
4/5                      0.8892      0.091      9.796      0.000       0.711       1.067
5/6                      1.1695      0.025     47.085      0.000       1.121       1.218
6/7                      0.9501      0.019     48.942      0.000       0.912       0.988
7/8                      0.8473      0.035     24.398      0.000       0.779       0.915
8/9                      1.6397      0.194      8.446      0.000       1.259       2.020
========================================================================================
"""

In [114]:
predicted = res_prob.model.predict(res_prob.params, exog = np.array(df_scaled_X_test)[:, None])

In [115]:
rowmax = np.amax(predicted[:,0], axis = 1)

In [116]:
# Need to change back to 3 to 9 scale
y_pred = []
for i, r in enumerate(rowmax):
    y_pred.append(list(predicted[i,0]).index(r)+3)

In [117]:
# Using the list comparison what is the percentage of matches
np.sum(y_test==y_pred)/len(y_test)

0.5361538461538462

In [157]:
cf = multilabel_confusion_matrix(y_test, y_pred)
l =['3', '4', '5', '6', '7', '8', '9']
mlstats(cf, l)

Label: 3, Precision: 1.0000, Recall: 0.9915, Specificity: 0.0000, Accuracy: 0.9915, F1: 0.9957 
Label: 4, Precision: 1.0000, Recall: 0.9700, Specificity: 1.0000, Accuracy: 0.9700, F1: 0.9848 
Label: 5, Precision: 0.8126, Recall: 0.8045, Specificity: 0.5901, Accuracy: 0.7377, F1: 0.8085 
Label: 6, Precision: 0.4743, Recall: 0.6772, Specificity: 0.5233, Accuracy: 0.5831, F1: 0.5579 
Label: 7, Precision: 0.9474, Recall: 0.8544, Specificity: 0.4184, Accuracy: 0.8215, F1: 0.8985 
Label: 8, Precision: 0.9992, Recall: 0.9723, Specificity: 0.0000, Accuracy: 0.9715, F1: 0.9856 
Label: 9, Precision: 1.0000, Recall: 0.9969, Specificity: 0.0000, Accuracy: 0.9969, F1: 0.9984 


In [ ]:
# This result is slightly better but in both cases the model under performs in mid-grade values. 
# Let's try the probit model

